## Import

In [1]:
from typing import List
from pydantic import BaseModel, ConfigDict
from crimson.types_beta.addon.pydantic import PydanticCompatible


## BaseModel Typecheck Failure

Let's prepare some classes first.

In [2]:
class MyType(List[int]):
    """
    My Custom
    """
    
class MyProps(BaseModel):
    my_arg: MyType

    model_config = ConfigDict(arbitrary_types_allowed=True)

If we annotate a field of the `BaseModel` using our custom type,

could we set `my_arg` using the innertype instance of our custom type?

In [3]:
try:
	MyProps(
		my_arg = [1, 2, 3]
	)
except Exception as e:
	print(e)

1 validation error for MyProps
my_arg
  Input should be an instance of MyType [type=is_instance_of, input_value=[1, 2, 3], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of


No, it is not possible.

Therefore, we use `PydanticCompatible`.

In [4]:
class MyCompatibleType(List[int], PydanticCompatible):
    """
    My Custom
    """
    
class MyProps(BaseModel):
    my_arg: MyCompatibleType
    
MyProps(
	my_arg = [1, 2, 3]
)


MyProps(my_arg=[1, 2, 3])

Now, our custom type behaves like `List[int]` in `BaseModel` without any error.

* Note that, we don't even add the `model_config = ConfigDict(arbitrary_types_allowed=True)`.


## Position of InnerType

In the default setting, we must place the innertype in the first place of the bases.

It is not always desirable. We sometimes want to place it in other positions for some reasons.

In that case, specify the positing by setting `validate_base_index`.

In [5]:
class MyCompatibleType(PydanticCompatible, List[int]):
    """
    My Custom
    """
    
    validate_base_index = 1

class MyProps(BaseModel):
    my_arg: MyCompatibleType
    
MyProps(
	my_arg = [1, 2, 3]
)

MyProps(my_arg=[1, 2, 3])

Otherwise, it will cause the error as below

In [6]:
class MyCompatibleType(PydanticCompatible, List[int]):
    """
    My Custom
    """


class MyProps(BaseModel):
    my_arg: MyCompatibleType

try:
	MyProps(
		my_arg = [1, 2, 3]
	)
except Exception as e:
    print(e)

The innertype of your custom type class must be placed as the first base of the type class as default. If you want to place it another position, specify the position using `validate_base_index`. See the link, https://github.com/crimson206/types/blob/main/example/addon/pydantic_compatible.ipynb
